# Family counts per year

By **Franklin Oliveira**

-----
This notebook contains all code necessary to make the "type" charts from `poliqueta` database. Here you'll find some basic data treatment and charts' code. 

Database: <font color='blue'>'IBUFRJ27.07.2020 - visualização.xlsx'</font>.

In [1]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# quick visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Waffle Charts
# from pywaffle import Waffle 
# docs: https://pywaffle.readthedocs.io/en/latest/examples/block_shape_distance_location_and_direction.html

# visualization
import altair as alt

# enabling notebook renderer
# alt.renderers.enable('notebook')
alt.renderers.enable('default')

# disabling rows limit
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

In [2]:
NewTable = pd.read_csv('./data/merged_db.csv', sep=';', encoding='utf-8-sig')

In [3]:
NewTable['family'] = NewTable['family'].astype(str)

# formatando a string NaN
NewTable['family'] = NewTable['family'].apply(lambda x: 'NaN' if x=='Nan' else x)
NewTable['type'] = NewTable['type'].apply(lambda x: x if str(x) != 'nan' else 'Non-type').astype(str)

<br>

<font size=5>**Color Palette**</font>
<!-- 

<div class='row' style='padding-top:20px;'>
    <div class='col-md-6'>
        <img src="./src/img1.jpg" width='400px'>
    </div>
    <div class='col-md-6'>
        <img src="./src/img2.jpg" width='400px'>
    </div>
</div>

<br>

A partir das imagens acima, selecionamos cores (centróides) para criar a paleta de cores. Foram elas: 
<ul>
    <li style='color:#3CA67F'><b> #3CA67F </b># verde</li>
    <li style='color:#7A9FBF'><b> #7A9FBF </b># azul</li>
    <li style='color:#D94814'><b> #D94814 </b># laranja</li>
    <li style='color:#D96236'><b> #D96236 </b># laranja 2</li>
    <li style='color:#F2B999'><b> #F2B999 </b># 'cor de pele'</li>
    <li style='color:#A66C4B'><b> #A66C4B </b># marrom 1</li>
    <li style='color:#732C02'><b> #732C02 </b># marrom 2</li>
</ul>

A partir das cores "centróides", utilizamos a ferramenta Color Crafter para selecionar diferentes "shades" e auxiliar para categorização em diferentes grupos sugeridos pela equipe de Poliquetas do Museu Nacional. 

<ul>
    <li style='color:#3CA67F'><b> Verde: </b> ['#daffef', '#bbebd3', '#9adabc', '#77c8a5', '#57b791', '#3ca67f', '#2a9670', '#238762', '#257a56']</li>
    <li style='color:#7A9FBF'><b> Azul: </b> ['#e7e5df', '#ccd2d8', '#b2c0d0', '#96afc8', '#7a9fbf', '#5d90b6', '#3c81ae', '#0673a4', '#00669a']</li>
    <li style='color:#D94814'><b> laranja: </b> ['#ffbd84', '#ffaa74', '#ff9760', '#ff814b', '#fc6b36', '#eb5824', '#d94814', '#c83b03', '#b73000']</li>
    <li style='color:#D96236'><b> laranja 2: ['#ffeba9', '#ffd391', '#ffbb7b', '#fda468', '#f18e56', '#e57846', '#d96236', '#cc4d28', '#bf381b']</b> </li>
    <li style='color:#F2B999'><b> cor de pele: ['#ffe9c3', '#fbd0ad', '#f2b999', '#e8a287', '#dd8c76', '#d27666', '#c76158', '#bb4d4b', '#ae393e']</b> </li>
    <li style='color:#A66C4B'><b> marrom 1: ['#d9c6af', '#ccad96', '#c1977c', '#b48061', '#a66c4b', '#975b39', '#874c2c', '#774124', '#683720']</b> </li>
    <li style='color:#732C02'><b> marrom 2: ['#eebd93', '#dfa47a', '#d28d60', '#c37746', '#b4622f', '#a3501d', '#92420e', '#823606', '#732c02']</b> </li>
</ul>



**Colors  (antigas):** 

<ul>
    <li style='color:#41A681'><b> #41A681 </b># verde1</li>
    <li style='color:#3CA67F'><b> #3CA67F </b># verde2</li>
    <li style='color:#7ACAAB'><b> #7ACAAB </b># verde claro</li>
    <li style='color:#78a1a1'><b> #78a1a1 </b># azul</li>
    <li style='color:#8ABFB0'><b> #8ABFB0 </b># azul claro</li>
    <li style='color:#FFB27C'><b> #FFB27C </b># cor de pele clara</li>
    <li style='color:#F29877'><b> #F29877 </b># cor de pele</li>
    <li style='color:#ed845e'><b> #ed845e </b># laranja claro1</li>
    <li style='color:#D96236'><b> #D96236 </b># laranja claro2</li>
    <li style='color:#D95323'><b> #D95323 </b># laranja 1</li>
    <li style='color:#D94B18'><b> #D94B18 </b># laranja 2</li>
    <li style='color:#D9C2AD'><b> #D9C2AD </b># bege</li>
    <li style='color:#A66C4B'><b> #A66C4B </b># marrom claro</li>
    <li style='color:#86471B'><b> #86471B </b># marrom1</li>
    <li style='color:#732C02'><b> #732C02 </b># marrom2</li>
    <li style='color:#592202'><b> #592202 </b># marrom escuro1</li>
    <li style='color:#3D1806'><b> #3D1806 </b># marrom escuro2</li>
    <li style='color:#0D0D0D'><b> #0D0D0D </b># preto</li>
</ul>

 -->

In [4]:
from src.MNViz_colors import *

<br>


## Graphs

---
### Creating chart: counts per order per year

In [5]:
orders = NewTable.groupby(['start_year','order', 'family']).count()['class'].reset_index().rename(columns={'class':'counts'})

orders.sort_values(['start_year','order'], inplace=True)  # ordering

In [6]:
# dropping remaining NaN's
orders = orders.dropna(subset=['order'])

In [8]:
# database
db = orders

# aux. variables
x_labels = db.sort_values('start_year')['start_year'].unique()
temp = db.groupby('order').sum().reset_index().sort_values('counts')
y_labels = temp['order'].unique()[::-1]
counts = db['counts'].unique()
counts = list(range(min(counts), max(counts), 50))

# selector
select_family = alt.selection_multi(fields=['family'], bind='legend')

g1 = alt.Chart(db, width=800, height=350, 
               title='Number of collected polychaetas per order each year').mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year',
             scale= alt.Scale(domain= x_labels)),
    y= alt.Y('order', type='nominal', title='Order',
             scale= alt.Scale(domain= y_labels),
             sort= alt.EncodingSortField(field='counts', op='sum', order='descending')),
    size = alt.Size('counts', title= 'Counts',
                    legend= alt.Legend(orient='right', direction='horizontal'),
                    scale=alt.Scale(domain= counts, range=[20,120])),
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color = alt.Color('family:N', title='Family', 
                      scale= alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                      legend= alt.Legend(columns=2, symbolLimit=42)),
    tooltip= alt.Tooltip(['start_year', 'counts'])
).add_selection(select_family).transform_filter(select_family)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# saving graph
# g1.save('./graphs/orders_per_year.html')
# g1

### number of polychaetas per family per year

In [9]:
teste = NewTable.groupby(['family','start_year']).count()['class'].reset_index().rename(
                                                                                    columns={'class':'counts'})

teste['start_year'] = teste['start_year'].astype(int)

<br>

**graph:** family per year

In [11]:
g1 = alt.Chart(teste,
               width=500, height=500, title='Number of collected polychaeta of each family per year').mark_circle(
                                                                                size=60).encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('family', type='nominal', title='Family',
            sort= alt.EncodingSortField(field='counts', op='count', order='descending')),
    size= alt.Size('counts', title='Count', scale=alt.Scale(range=[6,400])),
    tooltip = alt.Tooltip(['family', 'start_year', 'counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/families_per_year.html')
# g1

In [12]:
teste = NewTable.groupby(['family','order','start_year']).count()['class'].reset_index().rename(
                                                                                    columns={'class':'counts'})

teste['start_year'] = teste['start_year'].astype(int)

In [14]:
# database
db = teste

# aux. variables
x_labels = db.sort_values('start_year')['start_year'].unique()
temp = db.groupby('family').sum().reset_index().sort_values('counts')
y_labels = temp['family'].unique()[::-1]
counts = db['counts'].unique()
counts = list(range(min(counts), max(counts), 50))

# selector
select_family = alt.selection_multi(fields=['family'], bind='legend')

g1 = alt.Chart(db, width=500, height=800, 
        title='Number of collected animals of each family per year').mark_circle(size=60).encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year',
             scale= alt.Scale(domain=x_labels)),
    y= alt.Y('family', type='nominal', title='Family',
             scale= alt.Scale(domain=y_labels),
             sort= alt.EncodingSortField(field='counts', op='sum', order='descending')),
    size= alt.Size('counts', title='Count', 
                   legend= alt.Legend(orient='right', direction='horizontal'),
                   scale=alt.Scale(domain= counts,range=[20,120])),
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color = alt.Color('family:N', title='Family', 
                      scale= alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                      legend= alt.Legend(columns=2, symbolLimit=42)),
    tooltip = alt.Tooltip(['family', 'start_year', 'counts'])
).add_selection(select_family).transform_filter(select_family)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/families_per_year-colorful.html')
# g1

<br>

**The end!**

-----